<div style="float: left; width: 90%; height: auto; line-height: 200px; vertical-align: middle; background: #000099; padding: 0; margin: 0; border-top-left-radius: 10px; border-bottom-left-radius: 10px; border-top-right-radius: 10px; border-bottom-right-radius: 10px; font-family: Calibri; font-size: 50px; overflow: hidden; text-align: center;">
        <div style="text-align: center; width: 80%; margin: 0 auto; overflow: hidden; line-height: 80px; background-color:#000099">
                  <span id="dept_name" class="pretty_span">
                          <font color="white">**Parameter Grid**</font>
                  </span>
        </div>
</div>

<div style="float: left; width: 90%; height: auto; line-height: 200px; vertical-align: middle; background: #000000; padding: 0; margin: 0; border-top-left-radius: 10px; border-bottom-left-radius: 10px; border-top-right-radius: 10px; border-bottom-right-radius: 10px; font-family: Calibri; font-size: 30px; overflow: hidden; text-align: center;">
        <div style="text-align: center; width: 80%; margin: 0 auto; overflow: hidden; line-height: 50px; background-color:#000000">
                  <span id="dept_name" class="pretty_span">
                          <font color="white">**Table of contents**</font>
                  </span>
        </div>
</div>

## 1. [Import Libraries](#Import Libraries)
## 2. [Load the Dataset](#Load the Dataset)
## 3. [Create the Train and Test Datasets](#Create the Train and Test Datasets)
## 4. [Define Tokenizer with Vocab Size](#Define Tokenizer with Vocab Size)
## 5. [Create the Model](#Create the Model)
## 6. [Reproductibility and Model Definition](#Reproductibility and Model Definition)
## 7. [Create and Tune the Parameter Grip](#Create and Tune the Parameter Grip)
## 8. [Get the results of all Models and Find the Best One](#Get the results of all Models and Find the Best One)

# <a id='Import Libraries'></a> <u><font color='orange'>Import Libraries</font></u>

In [1]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.utils import np_utils
from keras.preprocessing.text import Tokenizer
from keras import metrics
from keras.layers.embeddings import Embedding
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import one_hot
from keras.layers import Flatten

from nltk import word_tokenize, pos_tag, chunk
from pprint import pprint
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer

from pprint import pprint
import pandas as pd
import numpy as np

from keras import optimizers
from keras.layers import Dense
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from keras.constraints import maxnorm
from keras.layers import Dropout
import os

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


# <a id='Load the Dataset'></a> <u><font color='orange'>Load the Dataset</font></u>

In [2]:
path = '/Users/leonidas/Documents/GitHub/Deep-Learning-Project/Results/'
filename = '25_cleaned_job_descriptions.csv'
data = pd.read_csv(os.path.join(path, filename), header = 0, names = ['Query', 'Description'])

# <a id='Create the Train and Test Datasets'></a> <u><font color='orange'>Create the Train and Test Datasets</font></u>

In [3]:
#Split data to train and test (80 - 20)

train, test = train_test_split(data, test_size = 0.2)

train_descs = train['Description']
train_labels = train['Query']
#train_labels = train['Job Title']
 
test_descs = test['Description']
test_labels = test['Query']

# <a id='Define Tokenizer with Vocab Size'></a> <u><font color='orange'>Define Tokenizer with Vocab Size</font></u>

In [4]:
num_labels = len(train_labels.unique().tolist())
vocab_size = 1000
 
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(train_descs)
 
x_train = tokenizer.texts_to_matrix(train_descs, mode='tfidf')
x_test = tokenizer.texts_to_matrix(test_descs, mode='tfidf')
 
encoder = LabelBinarizer()
encoder.fit(train_labels)
y_train = encoder.transform(train_labels)
y_test = encoder.transform(test_labels)

# <a id='Create the Model'></a> <u><font color='orange'>Create the Model</font></u>

In [5]:
##Parameter Grid Search for all parameters except for the neurons which are random numbers at this point.
def create_model(optimizer='adam', init_mode='uniform', activation='relu', dropout_rate=0.0, weight_constraint=0):
    model = Sequential()
    model.add(Dense(50, input_shape=(vocab_size,), activation = activation, kernel_initializer = init_mode, kernel_constraint=maxnorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, kernel_initializer = init_mode, activation= activation))
    model.add(Dropout(dropout_rate))
    model.add(Dense(num_labels))
    model.add(Activation('softmax'))
    
    # Compile model
    model.compile(loss = 'categorical_crossentropy',
              optimizer = optimizer, # or 'sgd'
              metrics = [metrics.categorical_accuracy, 'accuracy'])
    return model

# <a id='Reproductibility and Model Definition'></a> <u><font color='orange'>Reproductibility and Model Definition</font></u>

In [6]:
# Fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# Define the model
model = KerasClassifier(build_fn=create_model, verbose=0)

# <a id='Create and Tune the Parameter Grip'></a> <u><font color='orange'>Create and Tune the Parameter Grip</font></u>

In [9]:
##Parameter Tuning
optimizer = ['SGD', 'Adam']
init_mode = ['glorot_normal', 'glorot_uniform']
epochs = [30, 50]
batch_size = [16, 32]
activation = ['relu',  'sigmoid']
weight_constraint = [1, 2]
dropout_rate = [0.1, 0.2]
param_grid = dict(optimizer=optimizer, init_mode=init_mode, epochs=epochs, batch_size=batch_size, activation=activation, dropout_rate=dropout_rate, weight_constraint=weight_constraint)
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=1, refit=True, verbose=2)
grid_result = grid.fit(x_train, y_train)

Fitting 3 folds for each of 128 candidates, totalling 384 fits
[CV] activation=relu, batch_size=16, dropout_rate=0.1, epochs=30, init_mode=glorot_normal, optimizer=SGD, weight_constraint=1 
[CV]  activation=relu, batch_size=16, dropout_rate=0.1, epochs=30, init_mode=glorot_normal, optimizer=SGD, weight_constraint=1, total=  44.0s
[CV] activation=relu, batch_size=16, dropout_rate=0.1, epochs=30, init_mode=glorot_normal, optimizer=SGD, weight_constraint=1 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   44.7s remaining:    0.0s


[CV]  activation=relu, batch_size=16, dropout_rate=0.1, epochs=30, init_mode=glorot_normal, optimizer=SGD, weight_constraint=1, total=  43.8s
[CV] activation=relu, batch_size=16, dropout_rate=0.1, epochs=30, init_mode=glorot_normal, optimizer=SGD, weight_constraint=1 
[CV]  activation=relu, batch_size=16, dropout_rate=0.1, epochs=30, init_mode=glorot_normal, optimizer=SGD, weight_constraint=1, total=  42.0s
[CV] activation=relu, batch_size=16, dropout_rate=0.1, epochs=30, init_mode=glorot_normal, optimizer=SGD, weight_constraint=2 
[CV]  activation=relu, batch_size=16, dropout_rate=0.1, epochs=30, init_mode=glorot_normal, optimizer=SGD, weight_constraint=2, total=  43.8s
[CV] activation=relu, batch_size=16, dropout_rate=0.1, epochs=30, init_mode=glorot_normal, optimizer=SGD, weight_constraint=2 
[CV]  activation=relu, batch_size=16, dropout_rate=0.1, epochs=30, init_mode=glorot_normal, optimizer=SGD, weight_constraint=2, total=  44.9s
[CV] activation=relu, batch_size=16, dropout_rate=0

[CV]  activation=relu, batch_size=16, dropout_rate=0.1, epochs=50, init_mode=glorot_normal, optimizer=Adam, weight_constraint=1, total= 1.6min
[CV] activation=relu, batch_size=16, dropout_rate=0.1, epochs=50, init_mode=glorot_normal, optimizer=Adam, weight_constraint=2 
[CV]  activation=relu, batch_size=16, dropout_rate=0.1, epochs=50, init_mode=glorot_normal, optimizer=Adam, weight_constraint=2, total= 1.4min
[CV] activation=relu, batch_size=16, dropout_rate=0.1, epochs=50, init_mode=glorot_normal, optimizer=Adam, weight_constraint=2 
[CV]  activation=relu, batch_size=16, dropout_rate=0.1, epochs=50, init_mode=glorot_normal, optimizer=Adam, weight_constraint=2, total= 1.4min
[CV] activation=relu, batch_size=16, dropout_rate=0.1, epochs=50, init_mode=glorot_normal, optimizer=Adam, weight_constraint=2 
[CV]  activation=relu, batch_size=16, dropout_rate=0.1, epochs=50, init_mode=glorot_normal, optimizer=Adam, weight_constraint=2, total= 1.6min
[CV] activation=relu, batch_size=16, dropout

[CV]  activation=relu, batch_size=16, dropout_rate=0.2, epochs=30, init_mode=glorot_uniform, optimizer=SGD, weight_constraint=2, total= 1.0min
[CV] activation=relu, batch_size=16, dropout_rate=0.2, epochs=30, init_mode=glorot_uniform, optimizer=SGD, weight_constraint=2 
[CV]  activation=relu, batch_size=16, dropout_rate=0.2, epochs=30, init_mode=glorot_uniform, optimizer=SGD, weight_constraint=2, total=  55.2s
[CV] activation=relu, batch_size=16, dropout_rate=0.2, epochs=30, init_mode=glorot_uniform, optimizer=SGD, weight_constraint=2 
[CV]  activation=relu, batch_size=16, dropout_rate=0.2, epochs=30, init_mode=glorot_uniform, optimizer=SGD, weight_constraint=2, total=  50.0s
[CV] activation=relu, batch_size=16, dropout_rate=0.2, epochs=30, init_mode=glorot_uniform, optimizer=Adam, weight_constraint=1 
[CV]  activation=relu, batch_size=16, dropout_rate=0.2, epochs=30, init_mode=glorot_uniform, optimizer=Adam, weight_constraint=1, total=  54.6s
[CV] activation=relu, batch_size=16, dropo

...

[CV]  activation=sigmoid, batch_size=16, dropout_rate=0.2, epochs=50, init_mode=glorot_uniform, optimizer=SGD, weight_constraint=1, total= 2.9min
[CV] activation=sigmoid, batch_size=16, dropout_rate=0.2, epochs=50, init_mode=glorot_uniform, optimizer=SGD, weight_constraint=1 
[CV]  activation=sigmoid, batch_size=16, dropout_rate=0.2, epochs=50, init_mode=glorot_uniform, optimizer=SGD, weight_constraint=1, total= 3.5min
[CV] activation=sigmoid, batch_size=16, dropout_rate=0.2, epochs=50, init_mode=glorot_uniform, optimizer=SGD, weight_constraint=2 
[CV]  activation=sigmoid, batch_size=16, dropout_rate=0.2, epochs=50, init_mode=glorot_uniform, optimizer=SGD, weight_constraint=2, total= 3.9min
[CV] activation=sigmoid, batch_size=16, dropout_rate=0.2, epochs=50, init_mode=glorot_uniform, optimizer=SGD, weight_constraint=2 
[CV]  activation=sigmoid, batch_size=16, dropout_rate=0.2, epochs=50, init_mode=glorot_uniform, optimizer=SGD, weight_constraint=2, total= 3.5min
[CV] activation=sigmoid

[CV]  activation=sigmoid, batch_size=32, dropout_rate=0.1, epochs=30, init_mode=glorot_uniform, optimizer=Adam, weight_constraint=1, total= 1.5min
[CV] activation=sigmoid, batch_size=32, dropout_rate=0.1, epochs=30, init_mode=glorot_uniform, optimizer=Adam, weight_constraint=1 
[CV]  activation=sigmoid, batch_size=32, dropout_rate=0.1, epochs=30, init_mode=glorot_uniform, optimizer=Adam, weight_constraint=1, total= 1.4min
[CV] activation=sigmoid, batch_size=32, dropout_rate=0.1, epochs=30, init_mode=glorot_uniform, optimizer=Adam, weight_constraint=2 
[CV]  activation=sigmoid, batch_size=32, dropout_rate=0.1, epochs=30, init_mode=glorot_uniform, optimizer=Adam, weight_constraint=2, total= 1.7min
[CV] activation=sigmoid, batch_size=32, dropout_rate=0.1, epochs=30, init_mode=glorot_uniform, optimizer=Adam, weight_constraint=2 
[CV]  activation=sigmoid, batch_size=32, dropout_rate=0.1, epochs=30, init_mode=glorot_uniform, optimizer=Adam, weight_constraint=2, total= 1.4min
[CV] activation=

[CV]  activation=sigmoid, batch_size=32, dropout_rate=0.2, epochs=30, init_mode=glorot_normal, optimizer=SGD, weight_constraint=1, total= 1.7min
[CV] activation=sigmoid, batch_size=32, dropout_rate=0.2, epochs=30, init_mode=glorot_normal, optimizer=SGD, weight_constraint=1 
[CV]  activation=sigmoid, batch_size=32, dropout_rate=0.2, epochs=30, init_mode=glorot_normal, optimizer=SGD, weight_constraint=1, total= 1.4min
[CV] activation=sigmoid, batch_size=32, dropout_rate=0.2, epochs=30, init_mode=glorot_normal, optimizer=SGD, weight_constraint=2 
[CV]  activation=sigmoid, batch_size=32, dropout_rate=0.2, epochs=30, init_mode=glorot_normal, optimizer=SGD, weight_constraint=2, total= 1.7min
[CV] activation=sigmoid, batch_size=32, dropout_rate=0.2, epochs=30, init_mode=glorot_normal, optimizer=SGD, weight_constraint=2 
[CV]  activation=sigmoid, batch_size=32, dropout_rate=0.2, epochs=30, init_mode=glorot_normal, optimizer=SGD, weight_constraint=2, total= 1.7min
[CV] activation=sigmoid, batch

[CV]  activation=sigmoid, batch_size=32, dropout_rate=0.2, epochs=50, init_mode=glorot_normal, optimizer=Adam, weight_constraint=1, total= 2.5min
[CV] activation=sigmoid, batch_size=32, dropout_rate=0.2, epochs=50, init_mode=glorot_normal, optimizer=Adam, weight_constraint=1 
[CV]  activation=sigmoid, batch_size=32, dropout_rate=0.2, epochs=50, init_mode=glorot_normal, optimizer=Adam, weight_constraint=1, total= 2.8min
[CV] activation=sigmoid, batch_size=32, dropout_rate=0.2, epochs=50, init_mode=glorot_normal, optimizer=Adam, weight_constraint=2 
[CV]  activation=sigmoid, batch_size=32, dropout_rate=0.2, epochs=50, init_mode=glorot_normal, optimizer=Adam, weight_constraint=2, total= 2.8min
[CV] activation=sigmoid, batch_size=32, dropout_rate=0.2, epochs=50, init_mode=glorot_normal, optimizer=Adam, weight_constraint=2 
[CV]  activation=sigmoid, batch_size=32, dropout_rate=0.2, epochs=50, init_mode=glorot_normal, optimizer=Adam, weight_constraint=2, total= 2.8min
[CV] activation=sigmoid

[Parallel(n_jobs=1)]: Done 384 out of 384 | elapsed: 907.5min finished


# <a id='Get the results of all Models and Find the Best One'></a> <u><font color='orange'>Get the results of all Models and Find the Best One</font></u>

In [10]:
##Get all models results and find the best one.
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.133000 using {'activation': 'relu', 'batch_size': 16, 'dropout_rate': 0.1, 'epochs': 50, 'init_mode': 'glorot_normal', 'optimizer': 'Adam', 'weight_constraint': 2}
0.095750 (0.003726) with: {'activation': 'relu', 'batch_size': 16, 'dropout_rate': 0.1, 'epochs': 30, 'init_mode': 'glorot_normal', 'optimizer': 'SGD', 'weight_constraint': 1}
0.102125 (0.014167) with: {'activation': 'relu', 'batch_size': 16, 'dropout_rate': 0.1, 'epochs': 30, 'init_mode': 'glorot_normal', 'optimizer': 'SGD', 'weight_constraint': 2}
0.118500 (0.008455) with: {'activation': 'relu', 'batch_size': 16, 'dropout_rate': 0.1, 'epochs': 30, 'init_mode': 'glorot_normal', 'optimizer': 'Adam', 'weight_constraint': 1}
0.118500 (0.003429) with: {'activation': 'relu', 'batch_size': 16, 'dropout_rate': 0.1, 'epochs': 30, 'init_mode': 'glorot_normal', 'optimizer': 'Adam', 'weight_constraint': 2}
0.103875 (0.007170) with: {'activation': 'relu', 'batch_size': 16, 'dropout_rate': 0.1, 'epochs': 30, 'init_mode': 'glorot

In [15]:
##Parameter Grid for neurons in level 1 with all the previous parameters now fixed with the numbers from the previous best model.
def create_model(neurons=1):
    model = Sequential()
    model.add(Dense(neurons, input_shape=(vocab_size,), activation = 'relu', kernel_initializer = 'glorot_normal', kernel_constraint=maxnorm(2)))
    model.add(Dropout(0.1))
    model.add(Dense(1, kernel_initializer = 'glorot_normal', activation= 'relu'))
    model.add(Dropout(0.1))
    model.add(Dense(num_labels))
    model.add(Activation('softmax'))
    
    # Compile model
    model.compile(loss = 'categorical_crossentropy',
              optimizer = 'adam',
              metrics = [metrics.categorical_accuracy, 'accuracy'])
    return model

In [16]:
# Fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# Define the model
model = KerasClassifier(build_fn=create_model, batch_size=16, epochs=50, verbose=0)

In [21]:
##Parameter Tuning
neurons = [512, 1024, 2048]
param_grid = dict(neurons=neurons)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, refit=True, verbose=2)
grid_result = grid.fit(x_train, y_train)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] neurons=2048 ....................................................
[CV] ..................................... neurons=2048, total=12.4min
[CV] neurons=2048 ....................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 12.5min remaining:    0.0s


[CV] ..................................... neurons=2048, total=12.4min
[CV] neurons=2048 ....................................................
[CV] ..................................... neurons=2048, total=12.4min


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed: 37.5min finished


In [22]:
##Get all models results and find the best one.
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.061000 using {'neurons': 2048}
0.061000 (0.003348) with: {'neurons': 2048}


In [23]:
####Parameter Grid for neurons in level 2 with all the previous parameters now fixed.
def create_model(neurons=1):
    model = Sequential()
    model.add(Dense(512, input_shape=(vocab_size,), activation = 'relu', kernel_initializer = 'glorot_normal', kernel_constraint=maxnorm(2)))
    model.add(Dropout(0.1))
    model.add(Dense(neurons, kernel_initializer = 'glorot_normal', activation= 'relu'))
    model.add(Dropout(0.1))
    model.add(Dense(num_labels))
    model.add(Activation('softmax'))
    
    # Compile model
    model.compile(loss = 'categorical_crossentropy',
              optimizer = 'adam',
              metrics = [metrics.categorical_accuracy, 'accuracy'])
    return model

In [24]:
# Fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# Define the model
model = KerasClassifier(build_fn=create_model, batch_size=16, epochs=50, verbose=0)

In [25]:
##Parameter Tuning
neurons = [512, 1024, 2048]
param_grid = dict(neurons=neurons)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, refit=True, verbose=2)
grid_result = grid.fit(x_train, y_train)

Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV] neurons=512 .....................................................
[CV] ...................................... neurons=512, total= 7.3min
[CV] neurons=512 .....................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  7.4min remaining:    0.0s


[CV] ...................................... neurons=512, total= 6.8min
[CV] neurons=512 .....................................................
[CV] ...................................... neurons=512, total= 7.8min
[CV] neurons=1024 ....................................................
[CV] ..................................... neurons=1024, total= 9.7min
[CV] neurons=1024 ....................................................
[CV] ..................................... neurons=1024, total=12.7min
[CV] neurons=1024 ....................................................
[CV] ..................................... neurons=1024, total=15.0min
[CV] neurons=2048 ....................................................
[CV] ..................................... neurons=2048, total=10.0min
[CV] neurons=2048 ....................................................
[CV] ..................................... neurons=2048, total=10.6min
[CV] neurons=2048 ....................................................
[CV] .

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 94.8min finished


In [26]:
##Get all models results and find the best one.
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.557125 using {'neurons': 1024}
0.546625 (0.012304) with: {'neurons': 512}
0.557125 (0.004382) with: {'neurons': 1024}
0.531000 (0.000619) with: {'neurons': 2048}


In [27]:
##Parameter grid for the optimizer function with all other parameters fixed
def create_model(optimizer='adam'):
    model = Sequential()
    model.add(Dense(512, input_shape=(vocab_size,), activation = 'relu', kernel_initializer = 'glorot_normal', kernel_constraint=maxnorm(2)))
    model.add(Dropout(0.1))
    model.add(Dense(1024, kernel_initializer = 'glorot_normal', activation= 'relu'))
    model.add(Dropout(0.1))
    model.add(Dense(num_labels))
    model.add(Activation('softmax'))
    
    # Compile model
    model.compile(loss = 'categorical_crossentropy',
              optimizer = optimizer,
              metrics = [metrics.categorical_accuracy, 'accuracy'])
    return model

In [28]:
# Fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# Define the model
model = KerasClassifier(build_fn=create_model, batch_size=16, epochs=50, verbose=0)

In [29]:
##Parameter Tuning
optimizer = ['SGD']
param_grid = dict(optimizer=optimizer)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, refit=True, verbose=2)
grid_result = grid.fit(x_train, y_train)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] optimizer=SGD ...................................................
[CV] .................................... optimizer=SGD, total= 6.6min
[CV] optimizer=SGD ...................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  6.7min remaining:    0.0s


[CV] .................................... optimizer=SGD, total= 7.5min
[CV] optimizer=SGD ...................................................
[CV] .................................... optimizer=SGD, total= 7.0min


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed: 21.4min finished


In [30]:
##Get all models results and find the best one.
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.617625 using {'optimizer': 'SGD'}
0.617625 (0.004190) with: {'optimizer': 'SGD'}


In [31]:
##Parameter grid for the 1st Level neurons with all other parameters fixed(this time 2nd Level neurons=1024)
def create_model(neurons=1):
    model = Sequential()
    model.add(Dense(neurons, input_shape=(vocab_size,), activation = 'relu', kernel_initializer = 'glorot_normal', kernel_constraint=maxnorm(2)))
    model.add(Dropout(0.1))
    model.add(Dense(1024, kernel_initializer = 'glorot_normal', activation= 'relu'))
    model.add(Dropout(0.1))
    model.add(Dense(num_labels))
    model.add(Activation('softmax'))
    
    # Compile model
    model.compile(loss = 'categorical_crossentropy',
              optimizer = 'SGD',
              metrics = [metrics.categorical_accuracy, 'accuracy'])
    return model

In [32]:
# Fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# Define the model
model = KerasClassifier(build_fn=create_model, batch_size=16, epochs=50, verbose=0)

In [33]:
##Parameter Tuning
neurons = [1024, 2048]
param_grid = dict(neurons=neurons)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, refit=True, verbose=2)
grid_result = grid.fit(x_train, y_train)

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] neurons=1024 ....................................................
[CV] ..................................... neurons=1024, total= 9.7min
[CV] neurons=1024 ....................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  9.8min remaining:    0.0s


[CV] ..................................... neurons=1024, total= 8.7min
[CV] neurons=1024 ....................................................
[CV] ..................................... neurons=1024, total= 9.4min
[CV] neurons=2048 ....................................................
[CV] ..................................... neurons=2048, total=13.1min
[CV] neurons=2048 ....................................................
[CV] ..................................... neurons=2048, total=19.1min
[CV] neurons=2048 ....................................................
[CV] ..................................... neurons=2048, total=18.6min


[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed: 79.4min finished


In [34]:
##Get all models results and find the best one.
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.631125 using {'neurons': 2048}
0.621750 (0.005634) with: {'neurons': 1024}
0.631125 (0.004124) with: {'neurons': 2048}


In [35]:
##Parameter grid for batch_size numbers(searching for potentian improvment)
def create_model():
    model = Sequential()
    model.add(Dense(2048, input_shape=(vocab_size,), activation = 'relu', kernel_initializer = 'glorot_normal', kernel_constraint=maxnorm(2)))
    model.add(Dropout(0.1))
    model.add(Dense(1024, kernel_initializer = 'glorot_normal', activation= 'relu'))
    model.add(Dropout(0.1))
    model.add(Dense(num_labels))
    model.add(Activation('softmax'))
    
    # Compile model
    model.compile(loss = 'categorical_crossentropy',
              optimizer = 'SGD',
              metrics = [metrics.categorical_accuracy, 'accuracy'])
    return model

In [36]:
# Fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# Define the model
model = KerasClassifier(build_fn=create_model, epochs=50, verbose=0)

In [37]:
##Parameter Tuning
batch_size = [32, 64]
param_grid = dict(batch_size=batch_size)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, refit=True, verbose=2)
grid_result = grid.fit(x_train, y_train)

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] batch_size=32 ...................................................
[CV] .................................... batch_size=32, total= 9.4min
[CV] batch_size=32 ...................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  9.5min remaining:    0.0s


[CV] .................................... batch_size=32, total=12.0min
[CV] batch_size=32 ...................................................
[CV] .................................... batch_size=32, total= 8.7min
[CV] batch_size=64 ...................................................
[CV] .................................... batch_size=64, total= 5.4min
[CV] batch_size=64 ...................................................
[CV] .................................... batch_size=64, total= 4.7min
[CV] batch_size=64 ...................................................
[CV] .................................... batch_size=64, total= 4.8min


[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed: 45.5min finished


In [38]:
##Get all models results and find the best one.
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.633500 using {'batch_size': 32}
0.633500 (0.004691) with: {'batch_size': 32}
0.630375 (0.001017) with: {'batch_size': 64}


In [46]:
##Parameter grid for the 1st Level neurons(for the value 4096 only) with all other parameters fixed.
def create_model(neurons=1):
    model = Sequential()
    model.add(Dense(neurons, input_shape=(vocab_size,), activation = 'relu', kernel_initializer = 'glorot_normal', kernel_constraint=maxnorm(2)))
    model.add(Dropout(0.1))
    model.add(Dense(1024, kernel_initializer = 'glorot_normal', activation= 'relu'))
    model.add(Dropout(0.1))
    model.add(Dense(num_labels))
    model.add(Activation('softmax'))
    
    # Compile model
    model.compile(loss = 'categorical_crossentropy',
              optimizer = 'SGD',
              metrics = [metrics.categorical_accuracy, 'accuracy'])
    return model

In [47]:
# Fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# Define the model
model = KerasClassifier(build_fn=create_model, epochs=50, batch_size=32, verbose=0)

In [48]:
##Parameter Tuning
neurons = [4096]
param_grid = dict(neurons=neurons)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, refit=True, verbose=2)
grid_result = grid.fit(x_train, y_train)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] neurons=4096 ....................................................
[CV] ..................................... neurons=4096, total=12.7min
[CV] neurons=4096 ....................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 12.8min remaining:    0.0s


[CV] ..................................... neurons=4096, total=12.7min
[CV] neurons=4096 ....................................................
[CV] ..................................... neurons=4096, total=12.7min


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed: 38.4min finished


In [49]:
##Get all models results and find the best one.
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.635000 using {'neurons': 4096}
0.635000 (0.002783) with: {'neurons': 4096}
